In [1]:
import sys
print(sys.version)

3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]


In [2]:
import tensorflow as tf

model = tf.keras.models.load_model(r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_mobilenet.keras")
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 128, 128, 32)      │             864 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 128, 128, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 128, 128, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 128, 128, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 128, 128, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 128, 128, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 128, 128, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 128, 128, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 128, 128, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 128, 128, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 128, 128, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 129, 129, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 64, 64, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 38,167,127 (145.60 MB)

 Trainable params: 12,710,919 (48.49 MB)

 Non-trainable params: 34,368 (134.25 KB)

 Optimizer params: 25,421,840 (96.98 MB)

In [3]:
import cv2
import numpy as np
import os
import time
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

class_labels = ['chakri', 'gowthami', 'kavya','kumar','sai','uday','vijay'] 

def load_and_preprocess_image(img, target_size=(256, 256)):
    if img.shape[:2] != target_size:
        img = cv2.resize(img, target_size)
    img = np.expand_dims(img, axis=0)
    return img

model = load_model(r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_mobilenet.keras")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to grab frame or end of video.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    gg = frame.copy()
    for (x, y, w, h) in faces:
        cropped_face = gg[y:y+h, x:x+w]
        img = load_and_preprocess_image(cropped_face)
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions[0])
        predicted_class_name = class_labels[predicted_class]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        text = predicted_class_name
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        color = (255, 0, 0)  # Blue color
        thickness = 2
        
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = x + (w - text_size[0]) // 2  # Centering the text
        text_y = y + h + text_size[1] + 5  # Position slightly below the box

        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, thickness)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [3]:
import numpy as np
import os
import time
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

def load_and_preprocess_image(img_path, target_size=(256, 256)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_folder(model, folder_path, class_labels, target_size=(256, 256)):
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    start_time = time.time()
    for img_path in image_files:
        input_image = load_and_preprocess_image(img_path, target_size=target_size)
        predictions = model.predict(input_image)
        predicted_class = np.argmax(predictions[0])
        predicted_class_name = class_labels[predicted_class]
        # Print the predicted class name along with the image path
        print(f"Image: {os.path.basename(img_path)} | Predicted class: {predicted_class_name}")
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time to process {len(image_files)} images: {total_time:.2f} seconds")
    return total_time

In [4]:
model = load_model(r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_mobilenet.keras")
class_labels = ['chakri', 'gowthami', 'kavya','kumar','sai','uday','vijay'] 

folder_path = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\10"
time_10_images = predict_folder(model, folder_path, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Image: chakri (1).jpg | Predicted class: uday
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
Image: gowthami (2).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
Image: kumar (1).jpg | Predicted class: kumar
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step
Image: kumar (2).jpg | Predicted class: kumar
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
Image: Sri (1).jpg | Predicted class: uday
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step
Image: Sri (2).jpg | Predicted class: uday
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
Image: vijay (1).jpg | Predicted class: vijay
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
Image: vijay (2).jpg | Predicted class: vijay

Total time to process 10 images: 9.94 seconds


In [5]:
folder_path1 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\50"
time_50_images = predict_folder(model, folder_path1, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
Image: chakri (1).jpg | Predicted class: uday
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 591ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
Image: chakri (3).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
Image: chakri (6).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
Image: chakri (7).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
Image: chakri (8).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
Image: gowthami (16).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step
Image: gowthami (2).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
Image: gowthami (21).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step
Image: gowthami (3).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━

In [9]:
folder_path2 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\100"
time_100_images = predict_folder(model, folder_path2, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (1).jpg | Predicted class: uday
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (3).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (4).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (6).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (7).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (8).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: chakri (9).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: gowthami (10).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: gowthami (11).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image: gowthami (12).jpg | P

In [6]:
model1 = load_model(r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_resnet50v2.keras")
class_labels = ['chakri', 'gowthami', 'kavya','kumar','sai','uday','vijay'] 

resnet_10 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\10"
time_10_images = predict_folder(model1, resnet_10, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Image: chakri (1).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Image: gowthami (2).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Image: kumar (1).jpg | Predicted class: kumar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Image: kumar (2).jpg | Predicted class: kumar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Image: Sri (1).jpg | Predicted class: kavya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Image: Sri (2).jpg | Predicted class: kavya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Image: vijay (1).jpg | Predicted class: vijay
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Image: vijay (2).jpg | Predicted class: vijay

Total time to process 10 images: 5.85 seconds


In [11]:
resnet_50 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\50"
time_50_images = predict_folder(model1, resnet_50, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
Image: chakri (1).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Image: chakri (3).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Image: chakri (6).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Image: chakri (7).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Image: chakri (8).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Image: gowthami (16).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Image: gowthami (2).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Image: gowthami (21).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Image: gowthami (3).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━

In [12]:
resnet_100 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\100"
time_100_images = predict_folder(model1, resnet_100, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Image: chakri (1).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Image: chakri (3).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Image: chakri (4).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Image: chakri (6).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Image: chakri (7).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Image: chakri (8).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Image: chakri (9).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Image: gowthami (10).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Image: gowthami (11).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [13]:
model2 = load_model(r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_vgg16.keras")
class_labels = ['chakri', 'gowthami', 'kavya','kumar','sai','uday','vijay'] 

vgg_10 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\10"
time_10_images = predict_folder(model2, vgg_10, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: chakri (1).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
Image: gowthami (2).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
Image: kumar (1).jpg | Predicted class: kumar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
Image: kumar (2).jpg | Predicted class: kumar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
Image: Sri (1).jpg | Predicted class: kavya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
Image: Sri (2).jpg | Predicted class: kavya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Image: vijay (1).jpg | Predicted class: vijay
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
Image: vijay (2).jpg | Predicted class: vijay

Total time to process 10 images: 5.11 seconds


In [14]:
vgg_50 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\50"
time_50_images = predict_folder(model2, vgg_50, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Image: chakri (1).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Image: chakri (3).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
Image: chakri (6).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Image: chakri (7).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
Image: chakri (8).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
Image: gowthami (16).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
Image: gowthami (2).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
Image: gowthami (21).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
Image: gowthami (3).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━

In [15]:
vgg_100 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\100"
time_100_images = predict_folder(model2, vgg_100, class_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
Image: chakri (1).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
Image: chakri (2).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Image: chakri (3).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Image: chakri (4).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Image: chakri (6).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
Image: chakri (7).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
Image: chakri (8).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
Image: chakri (9).jpg | Predicted class: chakri
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
Image: gowthami (1).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
Image: gowthami (10).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
Image: gowthami (11).jpg | Predicted class: gowthami
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

---------------------------------- Keras to ONNX-------------------------

In [ ]:
!pip install tensorflow
!pip install onnx
!pip install tf2onnx

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]) # 2048 MB = 2GB
    except RuntimeError as e:
        print(e)

In [ ]:
import tensorflow as tf
import tf2onnx

model = tf.keras.models.load_model('/content/drive/MyDrive/Model_vgg16.keras')
onnx_model_path = '/content/drive/MyDrive/Model_vgg16.onnx'

spec = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=onnx_model_path)

print(f"Model saved to {onnx_model_path}")

In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import onnxruntime as ort
import time

class_labels = ['chakri', 'gowthami', 'kavya', 'kumar', 'sai', 'uday', 'vijay']

def load_onnx_model(onnx_model_path):
    session = ort.InferenceSession(onnx_model_path)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    return session, input_name, output_name

def predict_images_in_folder(session, input_name, output_name, image_folder, class_labels):
    batch_start_time = time.time()

    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for img_name in image_files:
        img_path = os.path.join(image_folder, img_name)

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(256, 256))
        input_tensor = image.img_to_array(img)
        input_tensor = np.expand_dims(input_tensor, axis=0).astype(np.float32)

        # Perform prediction
        start_time = time.time()
        output = session.run([output_name], {input_name: input_tensor})[0]
        end_time = time.time()

        prediction_time = end_time - start_time

        # Convert output to probabilities and get the predicted class
        probabilities = np.exp(output) / np.sum(np.exp(output), axis=1, keepdims=True)
        predicted_class_index = np.argmax(probabilities)

        # Print the result for each image
        print(f"Image: {img_name}")
        print(f"Predicted class: {class_labels[predicted_class_index]}")
        print(f"Time taken for prediction: {prediction_time:.6f} seconds")

    batch_end_time = time.time()
    total_batch_time = batch_end_time - batch_start_time

    print(f"\nTotal time for processing {len(image_files)} images: {total_batch_time:.6f} seconds")
    return total_batch_time

In [7]:
folders = {
    "10_images": r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\10",
    "50_images": r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\50",
    "100_images": r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\100"
}

onnx_model_path = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_mobilenet.onnx"
session, input_name, output_name = load_onnx_model(onnx_model_path)

for folder_name, folder_path in folders.items():
    print(f"\nProcessing folder: {folder_name}")
    predict_images_in_folder(session, input_name, output_name, folder_path, class_labels)


Processing folder: 10_images
Image: chakri (1).jpg
Predicted class: uday
Time taken for prediction: 0.019400 seconds
Image: chakri (2).jpg
Predicted class: chakri
Time taken for prediction: 0.016212 seconds
Image: gowthami (1).jpg
Predicted class: gowthami
Time taken for prediction: 0.024234 seconds
Image: gowthami (2).jpg
Predicted class: gowthami
Time taken for prediction: 0.013042 seconds
Image: kumar (1).jpg
Predicted class: kumar
Time taken for prediction: 0.026889 seconds
Image: kumar (2).jpg
Predicted class: kumar
Time taken for prediction: 0.016917 seconds
Image: Sri (1).jpg
Predicted class: uday
Time taken for prediction: 0.015072 seconds
Image: Sri (2).jpg
Predicted class: uday
Time taken for prediction: 0.032932 seconds
Image: vijay (1).jpg
Predicted class: vijay
Time taken for prediction: 0.025367 seconds
Image: vijay (2).jpg
Predicted class: vijay
Time taken for prediction: 0.021921 seconds

Total time for processing 10 images: 0.278489 seconds

Processing folder: 50_imag

In [8]:
onnx_model_path1 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_resnet50v2.onnx"
session, input_name, output_name = load_onnx_model(onnx_model_path1)

for folder_name, folder_path in folders.items():
    print(f"\nProcessing folder: {folder_name}")
    predict_images_in_folder(session, input_name, output_name, folder_path, class_labels)


Processing folder: 10_images
Image: chakri (1).jpg
Predicted class: chakri
Time taken for prediction: 0.158151 seconds
Image: chakri (2).jpg
Predicted class: chakri
Time taken for prediction: 0.080808 seconds
Image: gowthami (1).jpg
Predicted class: gowthami
Time taken for prediction: 0.072742 seconds
Image: gowthami (2).jpg
Predicted class: gowthami
Time taken for prediction: 0.068733 seconds
Image: kumar (1).jpg
Predicted class: kumar
Time taken for prediction: 0.074291 seconds
Image: kumar (2).jpg
Predicted class: kumar
Time taken for prediction: 0.065943 seconds
Image: Sri (1).jpg
Predicted class: kavya
Time taken for prediction: 0.071633 seconds
Image: Sri (2).jpg
Predicted class: kavya
Time taken for prediction: 0.070514 seconds
Image: vijay (1).jpg
Predicted class: vijay
Time taken for prediction: 0.063874 seconds
Image: vijay (2).jpg
Predicted class: vijay
Time taken for prediction: 0.076488 seconds

Total time for processing 10 images: 0.837449 seconds

Processing folder: 50_

In [9]:
onnx_model_path2 = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_vgg16.onnx"
session, input_name, output_name = load_onnx_model(onnx_model_path2)

for folder_name, folder_path in folders.items():
    print(f"\nProcessing folder: {folder_name}")
    predict_images_in_folder(session, input_name, output_name, folder_path, class_labels)


Processing folder: 10_images
Image: chakri (1).jpg
Predicted class: chakri
Time taken for prediction: 0.219598 seconds
Image: chakri (2).jpg
Predicted class: chakri
Time taken for prediction: 0.197888 seconds
Image: gowthami (1).jpg
Predicted class: gowthami
Time taken for prediction: 0.194309 seconds
Image: gowthami (2).jpg
Predicted class: gowthami
Time taken for prediction: 0.181840 seconds
Image: kumar (1).jpg
Predicted class: kumar
Time taken for prediction: 0.189461 seconds
Image: kumar (2).jpg
Predicted class: kumar
Time taken for prediction: 0.178932 seconds
Image: Sri (1).jpg
Predicted class: kavya
Time taken for prediction: 0.211938 seconds
Image: Sri (2).jpg
Predicted class: kavya
Time taken for prediction: 0.184517 seconds
Image: vijay (1).jpg
Predicted class: vijay
Time taken for prediction: 0.195044 seconds
Image: vijay (2).jpg
Predicted class: vijay
Time taken for prediction: 0.182601 seconds

Total time for processing 10 images: 1.987535 seconds

Processing folder: 50_